In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
import cv2
import tensorflow as tf

Using TensorFlow backend.
C:\Users\hp\anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\anaconda3\envs\PythonGPU\lib\sit

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14009957955787047055
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1459083673
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9443330959866670934
physical_device_desc: "device: 0, name: GeForce 930MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
x_train = np.load("x_train_200_no_val.npy")
y_train = np.load("y_train_200_no_val.npy")
x_train.shape, y_train.shape

((12000, 299, 299, 3), (12000,))

In [4]:
x_dev = np.load("x_test_200_no_val.npy")
y_dev = np.load("y_test_200_no_val.npy")
x_dev.shape, y_dev.shape

((3000, 299, 299, 3), (3000,))

In [5]:
input_shape = (299, 299, 3)

In [6]:
model = None
model = Sequential()

In [7]:
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))
    
model.compile(loss='binary_crossentropy',
            optimizer="adadelta",
            metrics=['accuracy'])

model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 299, 299, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 299, 299, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 149, 149, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________

In [8]:
model.fit(x=x_train, y=y_train, validation_data = (x_dev,y_dev), epochs=10, verbose=1, batch_size=4)


Train on 12000 samples, validate on 3000 samples
Epoch 1/10
12000/12000 [==============================] - 2086s 174ms/step - loss: 0.5637 - accuracy: 0.7998 - val_loss: 0.2112 - val_accuracy: 0.9330
Epoch 2/10
12000/12000 [==============================] - 2069s 172ms/step - loss: 0.2368 - accuracy: 0.9457 - val_loss: 0.1464 - val_accuracy: 0.9620
Epoch 3/10
12000/12000 [==============================] - 2069s 172ms/step - loss: 0.2124 - accuracy: 0.9498 - val_loss: 0.1519 - val_accuracy: 0.9587
Epoch 4/10
12000/12000 [==============================] - 2075s 173ms/step - loss: 0.1957 - accuracy: 0.9532 - val_loss: 0.1536 - val_accuracy: 0.9623
Epoch 5/10
12000/12000 [==============================] - 2072s 173ms/step - loss: 0.2121 - accuracy: 0.9503 - val_loss: 0.1332 - val_accuracy: 0.9637
Epoch 6/10
12000/12000 [==============================] - 2072s 173ms/step - loss: 0.2356 - accuracy: 0.9512 - val_loss: 0.1728 - val_accuracy: 0.9483
Epoch 7/10
12000/12000 [====================